In [1]:
import pandas as pd
import numpy as np
import math

from sklearn.metrics import mean_squared_error

In [2]:
train = pd.read_csv("train_subset_4.csv")

In [3]:
test = pd.read_csv("test.csv")

In [7]:
train.head(100)

,ego_id,u,v,t,x1,x2,x3
0,13,26,20,256.7,6.667426e-09,0.0,1.0
1,13,63,188,301.4,1.644699e-04,0.0,0.0
2,13,49,64,401.4,7.670612e-10,0.0,0.0
3,13,92,0,594.5,1.615412e-01,0.0,0.0
4,13,74,52,200.8,1.108634e-03,0.0,0.0
...,...,...,...,...,...,...,...
95,13,70,69,381.7,1.260792e+00,0.0,0.0
96,13,21,87,262.3,4.462837e-03,0.0,0.0
97,13,114,30,138.5,3.503281e-07,0.0,0.0
98,13,54,2,285.6,2.448323e-18,0.0,0.0


In [8]:
sub = pd.read_csv('submission.csv')

In [9]:
sub

,ego_id,u,v,x1
0,8,0,93,0.000000
1,8,0,143,0.000000
2,8,0,151,1.606742
3,8,1,24,0.026496
4,8,5,4,0.159857
...,...,...,...,...
810971,1709396984676,73,23,1.562170
810972,1709396984676,74,68,0.454210
810973,1709396984676,77,28,0.078504
810974,1709396984676,79,38,0.981812


In [8]:
train.shape

(30489281, 7)

In [4]:
attr = pd.read_csv("attr.csv")

In [5]:
attr

,ego_id,u,age,city_id,sex,school,university
0,0,227,68,-1,1,778293348,-1
1,0,45,38,237065842,1,82803468,238500268
2,0,142,60,237065842,1,196560139,-1
3,0,280,66,-1,2,963209731,720783270
4,0,41,18,-1,2,308862409,-1
...,...,...,...,...,...,...,...
14930743,1709396984692,2,16,492149712,2,769209871,-1
14930744,1709396984692,12,15,-1,1,-1,-1
14930745,1709396984692,18,23,-1,1,-1,-1
14930746,1709396984692,4,16,650683235,1,-1,-1


In [6]:
train_2 = pd.merge(train, attr, on=['ego_id', 'u']).merge(attr, right_on=['ego_id', 'u'], left_on=['ego_id', 'v'])

In [10]:
train_2.drop_duplicates()

,ego_id,u_x,v,t,x1,x2,x3,age_x,city_id_x,sex_x,school_x,university_x,u_y,age_y,city_id_y,sex_y,school_y,university_y
0,12,110,84,134.5,6.129875e-11,0.000000,0.0,39,954465843,2,223375057,590286237,84,30,221923457,2,62699071,-1
1,12,114,84,134.6,0.000000e+00,0.000000,0.0,45,-1,2,-1,183066311,84,30,221923457,2,62699071,-1
2,12,111,84,133.6,1.121872e-02,0.000000,0.0,43,117878396,2,-1,-1,84,30,221923457,2,62699071,-1
3,12,87,84,67.7,1.458379e-06,0.000000,0.0,39,6665424,1,-1,-1,84,30,221923457,2,62699071,-1
4,12,122,84,NaN,2.633960e-03,0.000000,0.0,62,-1,2,-1,732567033,84,30,221923457,2,62699071,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26776046,1709396984691,17,117,5.0,6.090294e+00,1.386294,0.0,18,-1,2,-1,-1,117,17,222975882,1,188621800,-1
26776047,1709396984691,3,1,NaN,0.000000e+00,3.044522,0.0,20,222975882,2,-1,-1,1,31,222975882,2,229683412,-1
26776048,1709396984691,101,136,2.8,3.822891e+00,0.000000,0.0,23,-1,2,-1,-1,136,22,-1,1,-1,-1
26776049,1709396984691,125,124,NaN,0.000000e+00,5.817111,1.0,21,222975882,1,5790347,-1,124,22,-1,2,-1,-1


In [11]:
train_2

,ego_id,u_x,v,t,x1,x2,x3,age_x,city_id_x,sex_x,school_x,university_x,u_y,age_y,city_id_y,sex_y,school_y,university_y
0,12,110,84,134.5,6.129875e-11,0.000000,0.0,39,954465843,2,223375057,590286237,84,30,221923457,2,62699071,-1
1,12,114,84,134.6,0.000000e+00,0.000000,0.0,45,-1,2,-1,183066311,84,30,221923457,2,62699071,-1
2,12,111,84,133.6,1.121872e-02,0.000000,0.0,43,117878396,2,-1,-1,84,30,221923457,2,62699071,-1
3,12,87,84,67.7,1.458379e-06,0.000000,0.0,39,6665424,1,-1,-1,84,30,221923457,2,62699071,-1
4,12,122,84,NaN,2.633960e-03,0.000000,0.0,62,-1,2,-1,732567033,84,30,221923457,2,62699071,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26776046,1709396984691,17,117,5.0,6.090294e+00,1.386294,0.0,18,-1,2,-1,-1,117,17,222975882,1,188621800,-1
26776047,1709396984691,3,1,NaN,0.000000e+00,3.044522,0.0,20,222975882,2,-1,-1,1,31,222975882,2,229683412,-1
26776048,1709396984691,101,136,2.8,3.822891e+00,0.000000,0.0,23,-1,2,-1,-1,136,22,-1,1,-1,-1
26776049,1709396984691,125,124,NaN,0.000000e+00,5.817111,1.0,21,222975882,1,5790347,-1,124,22,-1,2,-1,-1


In [12]:
def dis(x,y):
    return abs(x-y) if x > 0 and y > 0 else -1

In [13]:
un1 = train_2["university_x"].tolist()
un2 = train_2["university_y"].tolist()

In [14]:
def tmp(x, y):
    if x == -1 or y == -1:
        return -1
    return abs(x - y) 


train_2["city"] = list(map(tmp, train_2["city_id_x"], train_2["city_id_y"]))
train_2["university"] = list(map(tmp, train_2["university_x"], train_2["university_y"]))
train_2["sex"] = (train_2["sex_x"] + train_2["sex_y"]) % 2
train_2["school"] = list(map(tmp, train_2["school_y"], train_2["school_x"]))
train_2["age_dist"] = abs(train_2["age_x"] - train_2["age_y"])
train_2["age_mean"] = (train_2["age_x"] + train_2["age_y"]) / 2



In [28]:
def sex_cat(x, y):
    if  x == 1 and y == 1:
        return 1
    elif x == 2 and y == 2:
        return 2
    else:
        return 0
    
def is_the_same(x, y):
    if  x == y and x != -1:
        return 2
    elif x != -1 or y != -1:
        return 0
    else:
        return 1
    
def age_eps(x):
    if x <= 8:
        return 1
    else: 
        return 0


train_2['sex'] = list(map(sex_cat, train_2["sex_x"], train_2["sex_y"]))
train_2["city"] = list(map(is_the_same, train_2["city_id_x"], train_2["city_id_y"]))
train_2["university"] = list(map(is_the_same, train_2["university_x"], train_2["university_y"]))
train_2["school"] = list(map(is_the_same, train_2["school_y"], train_2["school_x"]))
train_2["gen"] = list(map(age_eps, train_2["age_dist"]))

In [29]:
train_3 = train_2.drop(["city_id_x", "city_id_y", "university_x", "university_y", "sex_x", "sex_y", "school_x", "school_y", "age_x", "age_y", "u_y"], axis=1)

In [30]:
train_3.rename(columns= {"u_x": "u"} , inplace=True)


In [31]:
train_3

,ego_id,u,v,t,x1,x2,x3,city,university,sex,school,age_dist,age_mean,gen
0,12,110,84,134.5,6.129875e-11,0.000000,0.0,0,0,2,0,9,34.5,0
1,12,114,84,134.6,0.000000e+00,0.000000,0.0,0,0,2,0,15,37.5,0
2,12,111,84,133.6,1.121872e-02,0.000000,0.0,0,1,2,0,13,36.5,0
3,12,87,84,67.7,1.458379e-06,0.000000,0.0,0,1,0,0,9,34.5,0
4,12,122,84,NaN,2.633960e-03,0.000000,0.0,0,0,2,0,32,46.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26776046,1709396984691,17,117,5.0,6.090294e+00,1.386294,0.0,0,1,0,0,1,17.5,1
26776047,1709396984691,3,1,NaN,0.000000e+00,3.044522,0.0,2,1,2,0,11,25.5,0
26776048,1709396984691,101,136,2.8,3.822891e+00,0.000000,0.0,1,1,0,1,1,22.5,1
26776049,1709396984691,125,124,NaN,0.000000e+00,5.817111,1.0,0,1,0,0,1,21.5,1


In [41]:
grouped_data1 = train_3.groupby(['ego_id', 'u']).agg({'v':'count'}).reset_index().rename(columns = {'v':'v_count'})

In [42]:
grouped_data = train_3.groupby(['ego_id', 'v']).agg({'u':'count'}).reset_index().rename(columns = {'u':'u_count'})

In [43]:
grouped_data

,ego_id,v,u_count
0,13,0,90
1,13,1,1
2,13,2,19
3,13,3,1
4,13,4,3
...,...,...,...
2646621,1709396984692,12,4
2646622,1709396984692,14,2
2646623,1709396984692,15,4
2646624,1709396984692,16,3


In [44]:
grouped_data = pd.merge(grouped_data1, grouped_data, left_on = ['ego_id', 'u'], right_on = ['ego_id', 'v'])

In [45]:
grouped_data['freinds'] = grouped_data['v_count']+grouped_data['u_count']
grouped_data

,ego_id,u,v_count,v,u_count,freinds
0,13,0,130,0,90,260
1,13,1,1,1,1,2
2,13,2,11,2,19,22
3,13,4,7,4,3,14
4,13,5,3,5,4,6
...,...,...,...,...,...,...
2451083,1709396984692,12,3,12,4,6
2451084,1709396984692,14,4,14,2,8
2451085,1709396984692,15,5,15,4,10
2451086,1709396984692,16,5,16,3,10


In [46]:
df = pd.merge(train_3, grouped_data[['ego_id', 'u', 'freinds']], on = ['ego_id', 'u'])
df = pd.merge(df, grouped_data[['ego_id', 'v', 'freinds']], on = ['ego_id', 'v'])
df = df.rename(columns = {'freinds_x':'freinds_u', 'freinds_y':'freinds_v'})
df

,ego_id,u,v,t,x1,x2,x3,city,university,sex,school,age_dist,age_mean,freinds_u,freinds_v
0,13,26,20,256.7,6.667426e-09,0.000000,1.0,0,-1,0,-1,2,29.0,32,32
1,13,0,20,144.7,1.536793e-04,0.000000,1.0,0,-1,0,0,5,32.5,260,32
2,13,148,20,NaN,0.000000e+00,0.000000,1.0,0,-1,0,0,19,39.5,16,32
3,13,2,20,79.1,6.092806e-07,0.000000,0.0,18712486,-1,0,84863200,5,32.5,22,32
4,13,17,20,338.1,1.138490e-08,0.000000,1.0,0,-1,0,0,8,34.0,44,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26006041,1709396984692,12,2,9.5,7.487368e-01,0.000000,0.0,-1,-1,1,-1,1,15.5,6,16
26006042,1709396984692,15,2,7.5,1.835345e+00,2.484907,0.0,-1,-1,0,378527603,4,18.0,10,16
26006043,1709396984692,0,18,NaN,0.000000e+00,2.484907,0.0,-1,-1,1,-1,4,21.0,28,2
26006044,1709396984692,0,10,0.9,1.993434e+00,2.302585,0.0,-1,-1,1,-1,3,17.5,28,8


In [47]:
df['u_v'] = (1 + df['x2'])*df['freinds_u'] + (1 + df['x3'])*df['freinds_v']

In [48]:
df

,ego_id,u,v,t,x1,x2,x3,city,university,sex,school,age_dist,age_mean,freinds_u,freinds_v,u_v
0,13,26,20,256.7,6.667426e-09,1.000000,2.0,0,-1,0,-1,2,29.0,32,32,96.000000
1,13,0,20,144.7,1.536793e-04,1.000000,2.0,0,-1,0,0,5,32.5,260,32,324.000000
2,13,148,20,NaN,0.000000e+00,1.000000,2.0,0,-1,0,0,19,39.5,16,32,80.000000
3,13,2,20,79.1,6.092806e-07,1.000000,1.0,18712486,-1,0,84863200,5,32.5,22,32,54.000000
4,13,17,20,338.1,1.138490e-08,1.000000,2.0,0,-1,0,0,8,34.0,44,32,108.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26006041,1709396984692,12,2,9.5,7.487368e-01,1.000000,1.0,-1,-1,1,-1,1,15.5,6,16,22.000000
26006042,1709396984692,15,2,7.5,1.835345e+00,3.484907,1.0,-1,-1,0,378527603,4,18.0,10,16,50.849066
26006043,1709396984692,0,18,NaN,0.000000e+00,3.484907,1.0,-1,-1,1,-1,4,21.0,28,2,99.577386
26006044,1709396984692,0,10,0.9,1.993434e+00,3.302585,1.0,-1,-1,1,-1,3,17.5,28,8,100.472383


In [49]:
from sklearn.ensemble import RandomForestRegressor

In [50]:
rgrs = RandomForestRegressor(max_depth = 5, random_state=42)

In [55]:
rgrs.fit(df[['x2', 'x3', 'sex', 'city', 'university', 'age_mean', 'u_v', 'school', 'age_dist']], df['x1'])

RandomForestRegressor(max_depth=5, random_state=42)

In [56]:
pred=rgrs.predict(df[['x2', 'x3', 'sex', 'city', 'university', 'age_mean', 'u_v', 'school', 'age_dist']])

In [57]:
from sklearn.metrics import mean_squared_error
mean_squared_error(df['x1'], pred)

1.0201155539899025

In [60]:
import pickle

In [62]:
# Предположим, что у вас есть объект модели с именем model
# Сохраняем модель на диск
with open('model.pkl', 'wb') as f:
    pickle.dump(rgrs, f)